In [8]:
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import gspread
from glob import glob
import pandas as pd
import numpy as np
import json
import sqlite3
import datetime
from dateutil.relativedelta import relativedelta
import re
import datetime


# ブドウシミュレーター
def grape_calculator_myfive(game, bb, rb, medals, cherry=True):
    bb_medals = 239.25
    rb_medals = 95.25
    replay_rate = 0.411
    if cherry:
        cherry_rate_high = 0.04228
    else:
        cherry_rate_high = 0.05847
    denominator_inner = (-medals - (game*3 - (bb*bb_medals + rb*rb_medals + game*replay_rate + game*cherry_rate_high))) / 8
    grape_rate = (game / denominator_inner) - ((game / denominator_inner) * 2)
    
    return grape_rate

def assign_area(unit_no, json_file_path):
    with open(json_file_path, "r", encoding="utf-8") as f:
        area_map = json.load(f)
    for rule in area_map:
        if rule["start"] <= unit_no <= rule["end"]:
            return rule["area"]
    return "その他"


def df_preprocessing(HALL_NAME, df, grape_calculator_myfive, assign_area):
    print(f'データ前処理を行います')
    df_pre = df.copy()
    df_pre["date"] = pd.to_datetime(df_pre["date"])
    df_pre.drop(columns=["result_id", "hall_id", "model_id"], inplace=True)
    df_pre = df_pre[["hall_name", "date", "model_name", "unit_no", "game", "BB", "RB", "medals"]]
    df_pre["BB_rate"] = (df_pre["game"] / df_pre["BB"]).round(1)
    df_pre["RB_rate"] = (df_pre["game"] / df_pre["RB"]).round(1)
    df_pre["Grape_rate"] = grape_calculator_myfive(df_pre["game"], df_pre["BB"], df_pre["RB"], df_pre["medals"], cherry=True).round(2)
    df_pre["Total_rate"] = (df_pre["game"] / (df_pre["BB"] + df_pre["RB"])).round(1)
    df_pre["month"] = df_pre["date"].dt.strftime("%Y-%m")
    df_pre["day"] = df_pre["date"].dt.day
    df_pre["weekday"] = df_pre["date"].dt.weekday
    df_pre["year"] = df_pre["date"].dt.year
    df_pre["unit_last"] = df_pre["unit_no"].astype(str).str[-1]

    JSON_FILE_PATH = f"C:/python/dataOnline/anaslo_02/json/{HALL_NAME}_area_map.json"
    # JSON_FILE_PATH = r"C:\python\dataOnline\anaslo_02\json\第一プラザ坂戸1000_area_map.json"
    df_pre["area"] = df_pre["unit_no"].apply(lambda x: assign_area(x, JSON_FILE_PATH))

    df_pre = df_pre.replace([np.inf, -np.inf], np.nan)
    df_pre = df_pre.fillna(0)

    model_list = list(df["model_name"].unique())
    print(f'以下のモデルが含まれています')
    for i, model in enumerate(model_list):
        print(f'{i}: {model}', end=", ")
    return df_pre


def create_df_from_database(HALL_NAME, DB_PATH):
    # Table name 取得
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
    tables = cursor.fetchall()
    # print(tables)

    cursor.execute(
    "SELECT hall_id, name FROM halls WHERE name LIKE ?", ("%" + HALL_NAME + "%",)
    )
    results = cursor.fetchall()

    # 結果表示
    if results:
        for hall_id, hall_name in results:
        # print(f" - hall_name: {hall_name}, hall_id: {hall_id}")
            print(f"🔍 '{HALL_NAME}' を含むホール名が見つかりました。")
    else:
        print(f"❌ '{HALL_NAME}' を含むホール名は見つかりませんでした。")

    query = """
        -- 出玉データにホール名と機種名を結合して取得
        SELECT
            r.*, 
            h.name AS hall_name,     -- ホール名を追加
            m.name AS model_name     -- 機種名を追加
        FROM results r
        JOIN halls h ON r.hall_id = h.hall_id  -- ホールと結合
        JOIN models m ON r.model_id = m.model_id  -- 機種と結合
        WHERE h.name = ?  -- 指定ホールのみ
        AND m.name LIKE '%ジャグラー%'  -- ジャグラー系機種に限定
        ORDER BY r.date DESC, r.unit_no ASC;
        """

    df = pd.read_sql_query(query, conn, params=(hall_name,))
    conn.close()
    print(f'データサイズ: {df.shape[0]} x {df.shape[1]}')
    
    return df


def create_pivot_table_date(
    df, start_date, end_date, day_targets, pivot_targets, index_targets, columns_targets
):
    """日付ごとにシートを作成して台ごとの出玉率一覧を作成"""
    df_filtered = df.copy()
    df_filtered = df_filtered[
        (df_filtered["date"].dt.date <= start_date)
        & (df_filtered["date"].dt.date >= end_date)
    ]
    df_filtered = df_filtered[df_filtered["day"] == day_targets]

    pivot_results = {}
    for col in pivot_targets:
        table = df_filtered.pivot_table(
            index=index_targets,
            columns=columns_targets,
            values=col,
            aggfunc="sum",
            margins=True,
            margins_name="Total",
        )
        pivot_results[col] = table.iloc[:, ::-1]

    game = pivot_results["game"]
    medals = pivot_results["medals"]
    rb = pivot_results["RB"]
    bb = pivot_results["BB"]
    rb_rate = (game / rb).round(1)
    total_rate = (game / (bb + rb)).round(1)
    medal_rate = ((medals + game * 3) / (game * 3)).round(3)

    labeled_tables = [
        ("GAME", game),
        ("MEDALS", medals),
        ("RB_RATE", rb_rate),
        ("TOTAL_RATE", total_rate),
        ("MEDAL_RATE", medal_rate),
        ("BB", bb),
        ("RB", rb),
    ]

    # ラベルを MultiIndex に付ける
    for label, df_table in labeled_tables:
        df_table.columns = pd.MultiIndex.from_product([[label], df_table.columns])

    # 列を交互に整列して統合・NaN除去
    interleaved_cols = [
        col
        for pair in zip(
            game.columns,
            medals.columns,
            bb.columns,
            rb.columns,
            medal_rate.columns,
            rb_rate.columns,
            total_rate.columns,
        )
        for col in pair
    ]

    merged = pd.concat([game, medals, medal_rate, bb, rb, rb_rate, total_rate], axis=1)[
        interleaved_cols
    ]
    merged.replace([np.inf, -np.inf, np.nan], None, inplace=True)

    return merged, game, medals, medal_rate, bb, rb, rb_rate, total_rate



# スプレッドシート認証設定
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
jsonf = r"C:\python\dataOnline\anaslo_02\json\spreeadsheet-347321-ff675ab5ccbd.json"
creds = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
client = gspread.authorize(creds)

In [ ]:
HALL_NAME = "EXA FIRST"
SPREADSHEET_ID = "10-B_vV1pvUzXmvGAiHhODGJgCloOsAmqSO9HvXpk_T8" # EXA FIRST
# HALL_NAME = "コンサートホールエフ成増"
# SPREADSHEET_ID = "1EDY2RfjDQNsapVrl2X-UrqPKoXrkQmYJnk3uPqccBxY"
# HALL_NAME = "第一プラザ坂戸1000"
# SPREADSHEET_ID = "170MVr-BB3LG-g5ItkDT-8TE6R68RW9zJhRfpvQiy-PE"
# HALL_NAME = "第一プラザ狭山店"


spreadsheet = client.open_by_key(SPREADSHEET_ID)
DB_PATH = r"C:\python\dataOnline\anaslo_02\db\anaslo_02.db"
df = create_df_from_database(HALL_NAME, DB_PATH)
df = df_preprocessing(HALL_NAME, df, grape_calculator_myfive, assign_area)
df.head(3)

🔍 'コンサートホールエフ成増' を含むホール名が見つかりました。
データサイズ: 30115 x 11
データ前処理を行います
以下のモデルが含まれています
0: マイジャグラーV, 1: ハッピージャグラーVIII, 2: アイムジャグラーEX-TP, 3: ジャグラーガールズ, 4: ウルトラミラクルジャグラー, 5: ファンキージャグラー2, 6: ゴーゴージャグラー3, 7: ミスタージャグラー, 

,hall_name,date,model_name,unit_no,game,BB,RB,medals,BB_rate,RB_rate,Grape_rate,Total_rate,month,day,weekday,year,unit_last,area
0,コンサートホールエフ成増,2025-05-16,マイジャグラーV,600,5521,28,19,1583,197.2,290.6,6.19,117.5,2025-05,16,4,2025,0,a: MYV
1,コンサートホールエフ成増,2025-05-16,マイジャグラーV,601,7269,28,27,936,259.6,269.2,5.71,132.2,2025-05,16,4,2025,1,a: MYV
2,コンサートホールエフ成増,2025-05-16,マイジャグラーV,602,5782,32,14,2355,180.7,413.0,5.72,125.7,2025-05,16,4,2025,2,a: MYV


## 出力データ
- 期間指定
    - 機種別の分析
    - 島別の分析
    - 台番号分析
    - 日付分析
    - 月分析
    - 過去n日の差枚比較
    - 全台系分析

## ピボットテーブル作成

In [3]:
def create_pivot_table(df, start_date, end_date, pivot_targets, index_targets, columns_targets, csv_path, hall_name=HALL_NAME):
    df_filtered = df.copy()
    df_filtered = df_filtered[
        (df_filtered["date"].dt.date <= start_date)
        & (df_filtered["date"].dt.date >= end_date)
    ]

    pivot_results = {}
    for col in pivot_targets:
        table = df_filtered.pivot_table(
            index=index_targets,
            columns=columns_targets,
            values=col,
            aggfunc="sum",
            margins=True,
            margins_name="total",
        )
        pivot_results[col] = table
    # pivot_results[col] = table.iloc[:, ::-1]

    game = pivot_results["game"]
    medals = pivot_results["medals"]
    rb = pivot_results["RB"]
    bb = pivot_results["BB"]
    rb_rate = (game / rb).round(1)
    total_rate = (game / (bb + rb)).round(1)
    medal_rate = ((medals+game*3) / (game*3)).round(3)

    labeled_tables = [
        ("GAME", game),
        ("MEDALS", medals),
        ("RB_RATE", rb_rate),
        ("TOTAL_RATE", total_rate),
        ("MEDAL_RATE", medal_rate),
        ("BB", bb),
        ("RB", rb),
    ]

    # ラベルを MultiIndex に付ける
    for label, df_table in labeled_tables:
        df_table.columns = pd.MultiIndex.from_product([[label], df_table.columns])

    # 列を交互に整列して統合・NaN除去
    interleaved_cols = [
        col
        for pair in zip(
            game.columns,
            medals.columns,
            bb.columns,
            rb.columns,
            medal_rate.columns,
            rb_rate.columns,
            total_rate.columns,
        )
        for col in pair
    ]

    merged = pd.concat([
        game, medals, medal_rate, bb, rb, rb_rate, total_rate], axis=1)[interleaved_cols]
    # merged.to_csv(csv_path)

    return merged, game, medals, medal_rate, bb, rb, rb_rate, total_rate

## MEDAL_RATE

In [4]:
today = datetime.date.today()
start_date = datetime.date.today()
end_date = start_date - relativedelta(months=6, days=start_date.day - 1)
print(f"対象期間: {start_date} から {end_date}")
print(f"対象ホール: {HALL_NAME}")

csv_path = f"{HALL_NAME}_medal_rate.csv"
pivot_targets = ["game", "medals", "BB", "RB"]
index_targets = ["area", "unit_no"]
colmns_targets = ["day"]
merged, game, medals, medal_rate, bb, rb, rb_rate, total_rate = create_pivot_table(
    df, start_date, end_date, pivot_targets, index_targets, colmns_targets, csv_path)
medal_rate.to_csv(csv_path)

target_rate = 1.05
medal_rate[("MEDAL_RATE", f"count_{target_rate}+")] = (medal_rate.iloc[:, :-1] >= target_rate).sum(axis=1)
countif = (medal_rate.iloc[:-1, :] >= target_rate).sum(axis=0)
medal_rate = pd.concat([medal_rate, pd.DataFrame([countif], index=[(f"count_{target_rate}+", "")])], axis=0)

rows, cols = medal_rate.shape
sheet_name = f"MEDAL_RATE"
try:
    worksheet = spreadsheet.worksheet(sheet_name)
    print(f"✅ シート「{sheet_name}」が既に存在します。")
except gspread.exceptions.WorksheetNotFound:
    worksheet = spreadsheet.add_worksheet(title=sheet_name, rows=str(rows+5), cols=str(cols+5))
    print(f"🆕 シート「{sheet_name}」を新規作成しました。")

sheet = spreadsheet.worksheet(sheet_name)
sheet.clear()
set_with_dataframe(sheet, medal_rate, include_index=True)
sheet.update_cell(1, 1, today.strftime("%Y-%m-%d UPDATED"))
print(f"✅ シート '{sheet_name}' に DataFrame を書き込みました！")

medal_rate.tail()


対象期間: 2025-05-17 から 2024-11-01
対象ホール: EXA FIRST
✅ シート「MEDAL_RATE」が既に存在します。
✅ シート 'MEDAL_RATE' に DataFrame を書き込みました！


MEDAL_RATE                                                   \
day                       1      2      3      4      5      6      7      8   
t: 1151-    1153      0.985  0.987  1.071  0.949  0.996  0.983  1.026  0.929   
            1154      1.020  0.987  1.002  0.914  1.000  1.034  0.959  1.028   
            1155      1.008  0.973  0.999  0.999  1.024  0.997  0.966  0.991   
total                 1.016  0.998  0.994  0.996  1.007  1.001  0.999  0.998   
count_1.05+          23.000  7.000  5.000  6.000  9.000  9.000  6.000  9.000   

                                ...                                        \
day                   9     10  ...     24      25      26      27     28   
t: 1151-    1153  1.016  0.971  ...  0.973   1.025   0.994   0.972  1.021   
            1154  0.916  0.912  ...  0.989   1.018   1.010   0.959  1.015   
            1155  1.006  0.957  ...  0.991   1.033   0.993   0.876  0.973   
total             0.991  0.995  ...  0.987   1.007   1.015   0.998  0.987   
count_1.05+       7.000  5.000  ...  6.000  15.000  23.000  14.000  5.000   

                                                            
day                  29      30      31  total count_1.05+  
t: 1151-    1153  1.037   0.981   0.972  1.002           2  
            1154  0.960   0.931   0.969  0.999           1  
            1155  1.078   1.014   1.022  0.996           1  
total             0.991   0.994   0.998  1.000           0  
count_1.05+       8.000  14.000  18.000  0.000          87  

[5 rows x 33 columns]

## 島分析

In [5]:
csv_path = f"{HALL_NAME}_area.csv"
pivot_targets = ["game", "medals", "BB", "RB"]
index_targets = ["area"]
colmns_targets = ["day"]
merged, game, medals, medal_rate, bb, rb, rb_rate, total_rate = create_pivot_table(
    df, start_date, end_date, pivot_targets, index_targets, colmns_targets, csv_path)
# medal_rate.to_csv(csv_path)
merged.head(3)

,GAME,MEDALS,BB,RB,MEDAL_RATE,RB_RATE,TOTAL_RATE,GAME,MEDALS,BB,...,MEDAL_RATE,RB_RATE,TOTAL_RATE,GAME,MEDALS,BB,RB,MEDAL_RATE,RB_RATE,TOTAL_RATE
day,1,1,1,1,1,1,1,2,2,2,...,31,31,31,total,total,total,total,total,total,total
area,,,,,,,,,,,,,,,,,,,,,
a: 1001-,384978,44788,1585,1200,1.039,320.8,138.2,253327,-6885,960,...,1.004,347.4,149.9,7927743,98186,30271,22935,1.004,345.7,149.0
b: 1011-,392273,22110,1499,1286,1.019,305.0,140.9,284390,-4995,1039,...,0.981,328.4,153.9,8141080,55672,30798,23966,1.002,339.7,148.7
c: 1021-,243764,39631,1013,797,1.054,305.9,134.7,169919,-9661,611,...,1.004,386.8,153.7,5020354,35052,18977,14219,1.002,353.1,151.2


## 台分析

In [6]:
csv_path = f"{HALL_NAME}_unit.csv"
pivot_targets = ["game", "medals", "BB", "RB"]
index_targets = ["area", "unit_no"]
colmns_targets = ["day"]
merged, game, medals, medal_rate, bb, rb, rb_rate, total_rate = create_pivot_table(
    df, start_date, end_date, pivot_targets, index_targets, colmns_targets, csv_path)
# medal_rate.to_csv(csv_path)
merged.head()
# for index in merged.index.get_level_values(0).unique():
#     merged_index = merged.xs(index, level="area", drop_level=False).iloc[:, 14:]
#     display(merged_index)

GAME MEDALS   BB   RB MEDAL_RATE RB_RATE TOTAL_RATE   GAME  \
day                   1      1    1    1          1       1          1      2   
area     unit_no                                                                
a: 1001- 1001     42311   6141  185  119      1.048   355.6      139.2  33660   
         1002     34168   4895  138  121      1.048   282.4      131.9  18471   
         1003     40333   7389  173  127      1.061   317.6      134.4  25252   
         1004     32844   1773  129   96      1.018   342.1      146.0  30988   
         1005     37114   4166  160   98      1.037   378.7      143.9  21611   

                 MEDALS   BB  ... MEDAL_RATE RB_RATE TOTAL_RATE    GAME  \
day                   2    2  ...         31      31         31   total   
area     unit_no              ...                                         
a: 1001- 1001     -2394  121  ...      0.963   308.6      155.7  900606   
         1002       428   72  ...      1.027   319.4      139.6  739077   
         1003     -2422   89  ...      0.934   459.4      182.3  738430   
         1004      1759  131  ...      1.007   371.4      155.1  749405   
         1005     -1150   77  ...      1.062   288.6      129.7  797780   

                 MEDALS    BB    RB MEDAL_RATE RB_RATE TOTAL_RATE  
day               total total total      total   total      total  
area     unit_no                                                   
a: 1001- 1001    -37472  3321  2519      0.986   357.5      154.2  
         1002     13261  2821  2159      1.006   342.3      148.4  
         1003     29101  2890  2198      1.013   336.0      145.1  
         1004     14521  2892  2161      1.006   346.8      148.3  
         1005      3191  2999  2317      1.001   344.3      150.1  

[5 rows x 224 columns]

# 日付、台番号指定
- SHEET_NAME : DAY1，DAY2，… DAY31

In [7]:
today = datetime.date.today()
start_date = datetime.date.today()
end_date = start_date - relativedelta(months=6, days=start_date.day - 1)
print(f"対象期間: {start_date} から {end_date} です")

pivot_targets = ["game", "medals", "BB", "RB"]
index_targets = ["area", "unit_no"]
columns_targets = ["date"]

today = datetime.date.today()
for day_target in range(today.day-1, today.day+1):
# for day_target in [2, 5, 7, 11, 15, 17, 22, 26]:
    merged, game, medals, medal_rate, bb, rb, rb_rate, total_rate = create_pivot_table_date(
        df, start_date, end_date, day_target, pivot_targets, index_targets, columns_targets)
    rows, cols = medal_rate.shape
    sheet_name = f"DAY{day_target}"
    try:
        worksheet = spreadsheet.worksheet(sheet_name)
        print(f"✅ シート「{sheet_name}」が既に存在します。")
    except gspread.exceptions.WorksheetNotFound:
        worksheet = spreadsheet.add_worksheet(title=sheet_name, rows=str(rows+5), cols=str(cols+5))
        print(f"🆕 シート「{sheet_name}」を新規作成しました。")
    sheet = spreadsheet.worksheet(sheet_name)
    sheet.clear()
    set_with_dataframe(sheet, merged, include_index=True)
    sheet.update_cell(1, 1, today.strftime("%Y-%m-%d : UPDATED"))
    print(f"✅ シート '{sheet_name}' に DataFrame を書き込みました！")

merged.head()

対象期間: 2025-05-17 から 2024-11-01 です
✅ シート「DAY16」が既に存在します。
✅ シート 'DAY16' に DataFrame を書き込みました！
✅ シート「DAY17」が既に存在します。
✅ シート 'DAY17' に DataFrame を書き込みました！


GAME MEDALS    BB    RB MEDAL_RATE RB_RATE TOTAL_RATE  \
date              Total  Total Total Total      Total   Total      Total   
area     unit_no                                                           
a: 1001- 1001     31164  -1862   110    92       0.98   338.7      154.3   
         1002     18709  -1404    62    61      0.975   306.7      152.1   
         1003     26821   4259   120    82      1.053   327.1      132.8   
         1004     19651    906    81    49      1.015   401.0      151.2   
         1005     26525   3175   113    74       1.04   358.4      141.8   

                                GAME              MEDALS                  BB  \
date             2025-04-17 00:00:00 2025-04-17 00:00:00 2025-04-17 00:00:00   
area     unit_no                                                               
a: 1001- 1001                   5488               -1382                  15   
         1002                    800                -626                   1   
         1003                   6994                2197                  38   
         1004                   1695                  38                   7   
         1005                   4432                 432                  19   

                  ...          MEDAL_RATE             RB_RATE  \
date              ... 2024-12-17 00:00:00 2024-12-17 00:00:00   
area     unit_no  ...                                           
a: 1001- 1001     ...               0.997               377.2   
         1002     ...                0.96               522.0   
         1003     ...               0.917              1188.0   
         1004     ...               1.061               333.6   
         1005     ...               1.032               438.6   

                          TOTAL_RATE                GAME              MEDALS  \
date             2024-12-17 00:00:00 2024-11-17 00:00:00 2024-11-17 00:00:00   
area     unit_no                                                               
a: 1001- 1001                  156.4                7168                1279   
         1002                  174.0                2390                -797   
         1003                  198.0                7890                2788   
         1004                  133.4                3384                -535   
         1005                  146.2                6029                 288   

                                  BB                  RB          MEDAL_RATE  \
date             2024-11-17 00:00:00 2024-11-17 00:00:00 2024-11-17 00:00:00   
area     unit_no                                                               
a: 1001- 1001                     31                  25               1.059   
         1002                      6                   6               0.889   
         1003                     42                  21               1.118   
         1004                     11                   7               0.947   
         1005                     25                  14               1.016   

                             RB_RATE          TOTAL_RATE  
date             2024-11-17 00:00:00 2024-11-17 00:00:00  
area     unit_no                                          
a: 1001- 1001                  286.7               128.0  
         1002                  398.3               199.2  
         1003                  375.7               125.2  
         1004                  483.4               188.0  
         1005                  430.6               154.6  

[5 rows x 49 columns]

In [122]:
# 各種ピボットテーブル
def create_pivot_tabel(df, model_name, start_date, end_date):
    df_filtered = df.copy()
    df_filtered = df_filtered[df_filtered["model_name"] == model_name]
    df_filtered = df_filtered[
        (df_filtered["date"].dt.date <= start_date)
        & (df_filtered["date"].dt.date >= end_date)
    ]
    pivot_targets = ["game", "medals", "RB_rate", "Total_rate", "BB", "RB"]
    pivot_results = {}
    for col in pivot_targets:
        table = df_filtered.pivot_table(
            index=["area", "model_name", "unit_no"],
            columns="date",
            values=col,
            aggfunc="sum",
        )
        # 日付列を反転・スライス
        # pivot_results[col] = table
        pivot_results[col] = table.iloc[:, ::-1]

    game = pivot_results["game"]
    medals = pivot_results["medals"]
    rb_rate = pivot_results["RB_rate"]
    total_rate = pivot_results["Total_rate"]
    rb = pivot_results["RB"]
    bb = pivot_results["BB"]
    # メダル累積3日間・7日間
    rolling7 = medals.iloc[:, ::-1].rolling(7, min_periods=7, axis=1).sum().iloc[:, ::-1]
    rolling3 = medals.iloc[:, ::-1].rolling(3, min_periods=3, axis=1).sum().iloc[:, ::-1]
    
    labeled_tables = [
        ("7ROLLING", rolling7),
        ("3ROLLING", rolling3),
        ("GAME", game),
        ("MEDALS", medals),
        ("RB_RATE", rb_rate),
        ("TOTAL_RATE", total_rate),
        ("BB", bb),
        ("RB", rb),
    ]

    # ラベルを MultiIndex に付ける
    for label, df_table in labeled_tables:
        df_table.columns = pd.MultiIndex.from_product([[label], df_table.columns])

    # 列を交互に整列して統合・NaN除去
    interleaved_cols = [
        col
        for pair in zip(
            rolling7.columns,
            rolling3.columns,
            game.columns,
            medals.columns,
            bb.columns,
            rb.columns,
            rb_rate.columns,
            total_rate.columns,
        )
        for col in pair
    ]

    merged = pd.concat([rolling7, rolling3, game, medals, bb, rb, rb_rate, total_rate], axis=1)[
        interleaved_cols
    ]

    return merged, game, medals, bb, rb, rb_rate, total_rate, rolling7, rolling3


today = datetime.date.today()
start_date = datetime.date.today()
end_date = start_date - relativedelta(days=14)
print(f"対象期間: {start_date} から {end_date}")

merged, game, medals, bb, rb, rb_rate, total_rate, rolling7, rolling3 = create_pivot_tabel(
    df, "マイジャグラーV", start_date, end_date
)
merged = merged[~(merged.iloc[:, 0].isna() | (merged.iloc[:, 0] == 0))]
merged.head()

# rolling3.head(10)

# rolling7 = medals.rolling(7, min_periods=7, axis=1).sum()
# rolling7.columns = rolling7.columns.set_levels(["ROLLING"], level=0)
# rolling7.head()

対象期間: 2025-05-12 から 2025-04-28


C:\Users\mkyag\AppData\Local\Temp\ipykernel_5236\2217143215.py:29: FutureWarning: Support for axis=1 in DataFrame.rolling is deprecated and will be removed in a future version. Use obj.T.rolling(...) instead
  rolling7 = medals.iloc[:, ::-1].rolling(7, min_periods=7, axis=1).sum().iloc[:, ::-1]
C:\Users\mkyag\AppData\Local\Temp\ipykernel_5236\2217143215.py:30: FutureWarning: Support for axis=1 in DataFrame.rolling is deprecated and will be removed in a future version. Use obj.T.rolling(...) instead
  rolling3 = medals.iloc[:, ::-1].rolling(3, min_periods=3, axis=1).sum().iloc[:, ::-1]


7ROLLING   3ROLLING       GAME     MEDALS  \
date                        2025-05-11 2025-05-11 2025-05-11 2025-05-11   
area     model_name unit_no                                               
c: 1021- マイジャグラーV   1021       -2109.0      465.0       8359       1759   
                    1022       -3404.0     -274.0       5547        582   
                    1023        -819.0    -1086.0       3562      -2021   
                    1024        1580.0      362.0       4594        456   
                    1025         633.0     3224.0       5848         50   

                                    BB         RB    RB_RATE TOTAL_RATE  \
date                        2025-05-11 2025-05-11 2025-05-11 2025-05-11   
area     model_name unit_no                                               
c: 1021- マイジャグラーV   1021            34         32      261.2      126.7   
                    1022            26          8      693.4      163.1   
                    1023             6         10      356.2      222.6   
                    1024            20          9      510.4      158.4   
                    1025            20         21      278.5      142.6   

                              7ROLLING   3ROLLING  ...    RB_RATE TOTAL_RATE  \
date                        2025-05-10 2025-05-10  ... 2025-04-29 2025-04-29   
area     model_name unit_no                        ...                         
c: 1021- マイジャグラーV   1021       -3697.0    -1950.0  ...      338.5      153.9   
                    1022       -1612.0    -1230.0  ...      717.1      201.7   
                    1023        1264.0      420.0  ...      307.5      135.9   
                    1024         898.0      232.0  ...     1303.0      325.8   
                    1025        -408.0     2742.0  ...      367.6      145.5   

                              7ROLLING   3ROLLING       GAME     MEDALS  \
date                        2025-04-28 2025-04-28 2025-04-28 2025-04-28   
area     model_name unit_no                                               
c: 1021- マイジャグラーV   1021           NaN        NaN       7514       -371   
                    1022           NaN        NaN       1202       -994   
                    1023           NaN        NaN       3497        -26   
                    1024           NaN        NaN       6144        206   
                    1025           NaN        NaN       2057       -509   

                                    BB         RB    RB_RATE TOTAL_RATE  
date                        2025-04-28 2025-04-28 2025-04-28 2025-04-28  
area     model_name unit_no                                              
c: 1021- マイジャグラーV   1021            27         22      341.5      153.3  
                    1022             1          1     1202.0      601.0  
                    1023            13          5      699.4      194.3  
                    1024            27         11      558.5      161.7  
                    1025             7          5      411.4      171.4  

[5 rows x 112 columns]

In [123]:
rolling7.head(10)

7ROLLING                                   \
date                        2025-05-11 2025-05-10 2025-05-09 2025-05-08   
area     model_name unit_no                                               
c: 1021- マイジャグラーV   1021       -2109.0    -3697.0    -3555.0    -4532.0   
                    1022       -3404.0    -1612.0     -988.0    -1708.0   
                    1023        -819.0     1264.0     1935.0     -577.0   
                    1024        1580.0      898.0      228.0      101.0   
                    1025         633.0     -408.0    -2017.0    -5426.0   
                    1026         -95.0       32.0     2341.0     1903.0   
d: 1027- マイジャグラーV   1027       -1127.0     -200.0     -491.0     -311.0   
                    1028       -1234.0    -2372.0    -3095.0    -3583.0   
                    1029       -2506.0    -2809.0    -2877.0    -2941.0   
                    1030         789.0     -782.0     -173.0     -614.0   

                                                                         \
date                        2025-05-07 2025-05-06 2025-05-05 2025-05-04   
area     model_name unit_no                                               
c: 1021- マイジャグラーV   1021       -5105.0    -9446.0    -9623.0    -6076.0   
                    1022        3872.0     3678.0     2675.0     3490.0   
                    1023        1426.0     1408.0     3035.0     1027.0   
                    1024        1319.0     2133.0      747.0     1509.0   
                    1025       -2726.0    -3788.0    -1106.0     -268.0   
                    1026        1492.0     1174.0     1574.0      418.0   
d: 1027- マイジャグラーV   1027         222.0     -626.0    -1647.0    -2338.0   
                    1028       -5559.0    -6500.0    -4653.0    -4388.0   
                    1029       -2294.0    -1214.0    -1932.0     -253.0   
                    1030       -3837.0    -1464.0    -1405.0     -511.0   

                                                                         \
date                        2025-05-03 2025-05-02 2025-05-01 2025-04-30   
area     model_name unit_no                                               
c: 1021- マイジャグラーV   1021           NaN        NaN        NaN        NaN   
                    1022           NaN        NaN        NaN        NaN   
                    1023           NaN        NaN        NaN        NaN   
                    1024           NaN        NaN        NaN        NaN   
                    1025           NaN        NaN        NaN        NaN   
                    1026           NaN        NaN        NaN        NaN   
d: 1027- マイジャグラーV   1027           NaN        NaN        NaN        NaN   
                    1028           NaN        NaN        NaN        NaN   
                    1029           NaN        NaN        NaN        NaN   
                    1030           NaN        NaN        NaN        NaN   

                                                   
date                        2025-04-29 2025-04-28  
area     model_name unit_no                        
c: 1021- マイジャグラーV   1021           NaN        NaN  
                    1022           NaN        NaN  
                    1023           NaN        NaN  
                    1024           NaN        NaN  
                    1025           NaN        NaN  
                    1026           NaN        NaN  
d: 1027- マイジャグラーV   1027           NaN        NaN  
                    1028           NaN        NaN  
                    1029           NaN        NaN  
                    1030           NaN        NaN

In [10]:
model_list = [
    "マイジャグラーV",
    # "ゴーゴージャグラー3",
    # "アイムジャグラーEX-TP",
    # "ファンキージャグラー2",
    # "ミスタージャグラー",
    # "ウルトラミラクルジャグラー",
    # "ハッピージャグラーVIII",
    # "ジャグラーガールズ",
]

start_date = datetime.date.today()
end_date = start_date - relativedelta(days=60)
print(f"{start_date} to {end_date}")

merged_models = pd.DataFrame()
for model in list(model_list):
    print(model)
    merged, game, medals, bb, rb, rb_rate, total_rate = create_pivot_tabel(df, model, start_date, end_date)
    for area in merged.index.get_level_values("area").unique():
        area_merged = merged.xs(area, level="area", drop_level=False)
        area_merged = area_merged[~area_merged.iloc[:, 0].isna()]
        empty_index = pd.MultiIndex.from_tuples([tuple([""] * area_merged.index.nlevels)], names=area_merged.index.names)
        empty_row = pd.DataFrame([[""] * area_merged.shape[1]], index=empty_index, columns=area_merged.columns)
        
        # if area_merged.shape[0]:
        #     summary_row = []
        #     for c in area_merged.columns:
        #         if c[0] in ["GAME", "MEDALS", "RB", "BB"]:
        #             game_sum = area_merged.get(("GAME", c[1]), pd.Series()).sum()
        #             summary_row.append(area_merged[c].sum())
        #         elif c[0] == "RB_RATE":
        #             game_sum = area_merged.get(("GAME", c[1]), pd.Series()).sum()
        #             rb_sum = area_merged.get(("RB", c[1]), pd.Series()).sum()
        #             rb_rate = game_sum / rb_sum if rb_sum != 0 else None
        #             summary_row.append(round(rb_rate, 1) if rb_rate else None)
        #         elif c[0] == "TOTAL_RATE":
        #             game_sum = area_merged.get(("GAME", c[1]), pd.Series()).sum()
        #             rb_sum = area_merged.get(("RB", c[1]), pd.Series()).sum()
        #             bb_sum = area_merged.get(("BB", c[1]), pd.Series()).sum()
        #             total_rate = game_sum / (rb_sum + bb_sum) if (rb_sum + bb_sum) != 0 else None
        #             summary_row.append(round(total_rate, 1) if total_rate else None)
            

            # summary_df = pd.DataFrame(summary_row).T
            # summary_df.index = pd.MultiIndex.from_tuples([tuple([""] * area_merged.index.nlevels)], names=area_merged.index.names)
            # summary_df.columns = area_merged.columns
        merged_models = pd.concat([merged_models, area_merged, empty_row])

merged_models = merged_models[~(merged_models.iloc[:, 0].isna() | (merged_models.iloc[:, 0] == 0))]
merged_models.to_csv(f"{HALL_NAME}_unit.csv")
    
merged_models.head(10)

2025-05-09 to 2025-03-10
マイジャグラーV


GAME MEDALS  BB  RB RB_RATE TOTAL_RATE  GAME  \
day                            31     31  31  31      31         31    30   
model_name area     unit_no                                                 
マイジャグラーV   a: 1001- 1001     7407    865  27  33   224.5      123.4  2920   
                    1002     6947   2003  32  26   267.2      119.8  8390   
                    1003     3610   -503  11  10   361.0      171.9  1553   
                    1004     1052   -438   3   1  1052.0      263.0  3155   
                    1005     9461   3612  44  40   236.5      112.6  6203   
                    1006     1037   -621   2   1  1037.0      345.7  7317   
                    1007     3913   -462  14   9   434.8      170.1  4843   
                    1008     6506    124  24  21   309.8      144.6  2808   
                    1009     6350   -253  24  14   453.6      167.1  1941   
                    1010     2981  -1429   5   9   331.2      212.9  5494   

                            MEDALS  BB  RB  ...  BB  RB RB_RATE TOTAL_RATE  \
day                             30  30  30  ...  2   2       2          2    
model_name area     unit_no                 ...                              
マイジャグラーV   a: 1001- 1001       144  12   6  ...  26  13   413.5      137.8   
                    1002       665  29  32  ...  12  18   193.7      116.2   
                    1003         3   6   4  ...   4   2   612.0      204.0   
                    1004     -1065   8   9  ...   2   2   431.0      215.5   
                    1005      -218  24  15  ...  34  25   270.0      114.4   
                    1006      -712  24  21  ...   5   5   269.2      134.6   
                    1007      -638  18   5  ...   9   4   485.0      149.2   
                    1008      -547   7   9  ...   6   2   948.5      237.1   
                    1009      -315   7   4  ...   3   3   408.0      204.0   
                    1010      -768  17  17  ...  14   6   612.0      183.6   

                             GAME MEDALS  BB  RB RB_RATE TOTAL_RATE  
day                            1      1   1   1       1          1   
model_name area     unit_no                                          
マイジャグラーV   a: 1001- 1001     6879    344  24  24   286.6      143.3  
                    1002     1441   -450   5   3   480.3      180.1  
                    1003     7691   3306  42  18   427.3      128.2  
                    1004     4512    226  19  11   410.2      150.4  
                    1005     1852   -162   9   1  1852.0      185.2  
                    1006     3912   -662  13  10   391.2      170.1  
                    1007     3854    -15  15   7   550.6      175.2  
                    1008     4801   -874  16   8   600.1      200.0  
                    1009     6071    276  23  16   379.4      155.7  
                    1010     5913   1100  26  18   328.5      134.4  

[10 rows x 186 columns]

## HISTORY
- 7DAYS_AGO_SUM
- 3DAYS_AGO_SUM
- RATE＿MEDAL
- MEDALS
- GAME
- RB_RATE
- TOTAL_RATE
- 